## Load & Preprocessing

### Load Libraries

In [4]:
import pandas as pd
import numpy as np
import pickle

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm

import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

from sentence_transformers import SentenceTransformer

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
import os

os.chdir("/content/drive/MyDrive/Research/Creativity/")

print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1Rjla_aYUBs_-hJKtkufVcqBOWmRjk2D_/Research/Creativity


### Wellbeing Index

In [6]:
hwbi = pd.read_excel('/content/drive/MyDrive/Research/Creativity/2010 County Health Rankings National Data_v2.xls',
                     sheet_name='Ranked Measure Data')

hwbi.columns = hwbi.iloc[0]
hwbi = hwbi[1:]

In [261]:
hwbi.head()

,FIPS,State,County,Unreliable,Deaths,Aggregate Population,YPLL Rate,95% CI - Low,95% CI - High,Quartile,...,Quartile,Unreliable,Zip Codes with Healthy Food,# Zip Codes,% Healthy Food,Quartile,Population,Liquor Stores,Liquor Store Rate,Quartile
1,01001,Alabama,Autauga,NaN,670,137881,9778.1,8785.9,10770.2,2,...,1,NaN,2,8,25,4,49109,2,0.407257,2
2,01003,Alabama,Baldwin,NaN,2148,449589,8221.7,7715.9,8727.4,1,...,4,NaN,11,26,42.307692,2,168233,13,0.772738,3
3,01005,Alabama,Barbour,NaN,424,79450,10686.1,9262.8,12109.4,2,...,1,NaN,2,5,40,2,28125,3,1.066667,4
4,01007,Alabama,Bibb,NaN,373,60430,13069.5,11270.6,14868.3,4,...,1,NaN,4,8,50,1,21341,1,0.468582,2
5,01009,Alabama,Blount,NaN,787,155580,8929.7,8040.2,9819.1,1,...,4,NaN,4,7,57.142857,1,55811,NaN,0,1


In [1]:
def correlation(df, indices):
  """ Returns correlation with Gallup data on state level """
  for index in indices:
    hwbi[index] = hwbi[index].astype(float)
    hwbi[index] = (hwbi[index] - hwbi[index].mean()) / hwbi[index].std()

  uw_index = 0
  for index in indices:
    uw_index += hwbi[index]
  hwbi['uw_index'] = uw_index / len(indices)

  gallup = pd.read_excel("/content/drive/MyDrive/Tony/50 states happiness/Responsibilism/Gallup/Talhelm's Report.xlsx")

  grouped = df.groupby('State')['uw_index'].mean()
  grouped = grouped.reset_index()

  grouped['uw_index'] = -grouped['uw_index']

  corr_df = pd.merge(gallup, grouped, on='State', how='left')
  corr = corr_df[['Well_Being_Index', 'uw_index']].corr()

  print(corr)

In [7]:
correlation(hwbi, ['Mentally Unhealthy Days', 'Physically Unhealthy Days'])

                  Well_Being_Index  uw_index
Well_Being_Index          1.000000  0.629779
uw_index                  0.629779  1.000000


### Tweets

In [232]:
tweets = pd.read_csv(
    "tweets.txt",
    sep="\t",
    header=None,
    names=["UserID", "TweetID", "Tweet", "CreatedAt"],
    on_bad_lines='skip'
)

In [ ]:
users = pd.read_csv(
    "users.txt",
    sep="\t",
    header=None,
    names=["UserID", "UserLocation"],
    on_bad_lines='skip',
    encoding='latin-1'
)

In [ ]:
tweets

,UserID,TweetID,Tweet,CreatedAt
0,22077441,10538487904,Ok today I have to find something to wear for ...,2010-03-15 17:35:58
1,22077441,10536835844,I am glad I'm having this show but I can't wai...,2010-03-15 16:53:44
2,22077441,10536809086,Honestly I don't even know what's going on any...,2010-03-15 16:52:59
3,22077441,10534149786,@LovelyJ_Janelle hey sorry I'm sitting infront...,2010-03-15 15:42:07
4,22077441,10530203659,Sitting infront of this sewing machine ... I d...,2010-03-15 13:55:22
...,...,...,...,...
5135944,73016661,8082730603,@NYCsocialit3 nun u,2010-01-22 14:14:30
5135945,73016661,8081316956,RT @BonesThisWorld: A trip too harlem for a fe...,2010-01-22 13:28:51
5135946,73016661,8076088351,@Uniquehope was good,2010-01-22 10:53:13
5135947,73016661,8075627082,@DaStar6MFM lmaooo yea dat shit is crazy,2010-01-22 10:40:05


In [ ]:
users

,UserID,UserLocation
0,22077441,"UT: 43.009815,-83.710408"
1,17145858,"UT: 38.856217,-90.056057"
2,33931268,"UT: 42.52548,-73.770351"
3,19521541,"UT: 40.735051,-74.228224"
4,26976263,"UT: 34.096153,-118.368732"
...,...,...
5131,61333491,"UT: 41.275229,-72.967524"
5132,20295678,"UT: 42.351076,-71.091274"
5133,23027707,"UT: 34.057372,-117.714922"
5134,35814058,"UT: 41.986132,-88.065072"


In [ ]:
users['UserLocation'] = users['UserLocation'].str.replace("UT: ", "")
users['latitude'] = users['UserLocation'].apply(lambda x: str(x).split(',')[0])
users['longitude'] = users['UserLocation'].apply(lambda x: str(x).split(',')[1] if len(str(x).split(',')) > 1 else None)

In [ ]:
import requests

def get_fips(lat, lon):
  r = requests.get(f'https://geo.fcc.gov/api/census/block/find?latitude={lat}&longitude={lon}&censusYear=2010&format=json')
  # print(r.json())
  return r.json()

users['fips_json'] = users.apply(lambda x: get_fips(x['latitude'], x['longitude']), axis=1)
users['FIPS'] = users['fips_json'].apply(lambda x: x['County']['FIPS'] if isinstance(x, dict) and 'County' in x else None)
users.to_csv("users_location_fips_record.csv")

In [224]:
users

,UserID,UserLocation,latitude,longitude,fips_json,FIPS
0,22077441,"43.009815,-83.710408",43.009815,-83.710408,{'messages': ['FCC0001: The coordinate lies on...,26049
1,17145858,"38.856217,-90.056057",38.856217,-90.056057,"{'Block': {'FIPS': '171194019012012', 'bbox': ...",17119
2,33931268,"42.52548,-73.770351",42.52548,-73.770351,"{'Block': {'FIPS': '360010143011000', 'bbox': ...",36001
3,19521541,"40.735051,-74.228224",40.735051,-74.228224,{'messages': ['FCC0001: The coordinate lies on...,34013
4,26976263,"34.096153,-118.368732",34.096153,-118.368732,"{'Block': {'FIPS': '060377002005001', 'bbox': ...",06037
...,...,...,...,...,...,...
5131,61333491,"41.275229,-72.967524",41.275229,-72.967524,"{'Block': {'FIPS': '090091546004009', 'bbox': ...",09009
5132,20295678,"42.351076,-71.091274",42.351076,-71.091274,{'messages': ['FCC0001: The coordinate lies on...,25025
5133,23027707,"34.057372,-117.714922",34.057372,-117.714922,"{'Block': {'FIPS': '060710003031000', 'bbox': ...",06071
5134,35814058,"41.986132,-88.065072",41.986132,-88.065072,"{'Block': {'FIPS': '170438410031016', 'bbox': ...",17043


In [233]:
tweets = tweets.dropna(subset=['UserID'])
tweets = tweets.drop(tweets[tweets['UserID'].str.contains(r'\D+')].index)

tweets['UserID'] = tweets['UserID'].astype(str)
users['UserID'] = users['UserID'].astype(str)

tweets = tweets.merge(users[['UserID', 'FIPS']], on='UserID', how='left')

tweets.to_csv("tweets_location_fips_record.csv")

In [279]:
tweets = tweets.merge(hwbi[['FIPS', 'uw_index']], on='FIPS', how='left')
tweets.to_csv("tweets_location_wellbeing.csv")

## Topic Modelling

### Preprocessing

In [ ]:
# This preprocessing code is adapted from HW1

def preprocess_corpus(data):
    """ Tokenize, sentencize, and lowercase the input data.

        Args:
            data: list of strings, where each string is a document

        Return:
            list of lists, with tokenized, lowercased sentences from each of the processed rows
    """
    tokenized_sentences = []

    for doc in data:
        # Split reviews into sentences
        if not isinstance(doc, str):
          continue
        sentences = sent_tokenize(doc)
        for sentence in sentences:
            # Tokenize and preprocess each sentence
            tokenized_sentences.append(word_tokenize(sentence.lower()))
    return tokenized_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
# Here I follow the preprocessing suggestions from Gensim: https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html

def additional_preprocessing(docs):
  return [
    [
      lemmatizer.lemmatize(token.lower())
      for token in doc
      if len(token) > 1 and not token.isnumeric() and token.lower() not in stop_words
    ]
    for doc in docs
  ]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
docs = {}

for group in tweets.groupby('FIPS'):
  FIPS = group[0]
  docs[FIPS] = group[1]['Tweet'].tolist()

In [ ]:
for key, doc in tqdm(docs.items()):
  docs[key] = preprocess_corpus(doc)

100%|██████████| 682/682 [11:11<00:00,  1.01it/s]


In [9]:
for key, doc in tqdm(docs.items()):
  docs[key] = additional_preprocessing(doc)

100%|██████████| 682/682 [02:53<00:00,  3.92it/s]


In [10]:
with open('docs_preprocessed.pickle', 'wb') as handle:
  pickle.dump(docs, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Calculate Distribution

In [30]:
with open('docs_preprocessed.pickle', 'rb') as handle:
  docs = pickle.load(handle)

In [31]:
cond_prob = pd.read_csv('/content/drive/MyDrive/Research/Creativity/facebook_topics-master/csv/wwbpFBtopics_condProb.csv')

In [41]:
lst_docs = list(docs.values())

for i, county_doc in enumerate(lst_docs):
  concat_doc = []
  for _, sentence in enumerate(county_doc):
    concat_doc += sentence
  lst_docs[i] = concat_doc

corpus = [' '.join(text) for text in lst_docs]

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [43]:
vocab_dct = vectorizer.vocabulary_

topic_dct = {i: {} for i in range(2000)}

for _, row in cond_prob.iterrows():
  word = row['term']
  topic = row['category']
  if word in vocab_dct:
    topic_dct[topic][vocab_dct[word]] = row['weight']

In [ ]:
num_docs = X.shape[0]
num_topics = len(topic_dct)
topic_doc_probs = np.zeros((num_docs, num_topics))

X_array = X.toarray()

for topic, word_probs in topic_dct.items():
  word_indices = np.array(list(word_probs.keys()))
  word_weights = np.array(list(word_probs.values()))

  word_frequencies = X_array[:, word_indices]

  topic_doc_probs[:, topic] = (word_frequencies / X_array.sum(axis=1, keepdims=True)) @ word_weights

topic_doc_probs_df = pd.DataFrame(topic_doc_probs, columns=topic_dct.keys())


In [ ]:
features_df = pd.merge(topic_doc_probs_df, hwbi[['FIPS', 'uw_index']], on='FIPS', how='left')

In [54]:
features_df

,0,1,2,3,4,5,6,7,8,9,...,1992,1993,1994,1995,1996,1997,1998,1999,FIPS,uw_index
0,0.000191,0.000747,0.000632,0.000139,0.000491,0.000605,0.000296,0.000274,0.000325,0.000434,...,0.000351,0.000233,0.000119,0.000326,0.000363,0.000091,0.000076,0.000344,1015,4.84
1,0.000044,0.000490,0.000161,0.000257,0.000345,0.000282,0.000521,0.000792,0.000242,0.000186,...,0.000131,0.000123,0.000141,0.000067,0.000332,0.000108,0.000061,0.000062,1039,3.17
2,0.000408,0.000488,0.000266,0.000206,0.000137,0.000304,0.000353,0.000392,0.000405,0.000459,...,0.000343,0.000139,0.000266,0.000191,0.000215,0.000217,0.000231,0.000360,1049,4.90
3,0.000131,0.000554,0.000270,0.000175,0.000280,0.000527,0.000135,0.000119,0.000322,0.000405,...,0.000314,0.000199,0.000264,0.000316,0.000307,0.000225,0.000142,0.000141,1073,3.53
4,0.000109,0.000437,0.000252,0.000185,0.000144,0.000482,0.000212,0.000247,0.000255,0.000188,...,0.000450,0.000251,0.000132,0.000229,0.000693,0.000246,0.000094,0.000082,1081,4.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,0.000124,0.000604,0.000284,0.000116,0.000378,0.000507,0.000328,0.000347,0.000408,0.000534,...,0.000225,0.000166,0.000156,0.000349,0.000419,0.000145,0.000076,0.000264,55131,2.67
678,0.000152,0.000216,0.000078,0.000159,0.000172,0.000666,0.000112,0.000104,0.000505,0.000275,...,0.000139,0.000197,0.000153,0.000076,0.000081,0.000323,0.000043,0.000037,55133,2.69
679,0.000095,0.000230,0.000318,0.000294,0.000341,0.000314,0.000562,0.000124,0.000272,0.000145,...,0.000305,0.000135,0.000472,0.000185,0.000438,0.000414,0.000107,0.000288,56001,2.96
680,0.000098,0.000825,0.000168,0.000043,0.000223,0.000235,0.000265,0.000317,0.000434,0.000349,...,0.000987,0.000215,0.000289,0.000430,0.000522,0.000120,0.000098,0.000070,56021,2.88


In [210]:
features_df.to_csv('features_df.csv')

In [55]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

cleaned = features_df.dropna(subset=['uw_index'])

X = cleaned.iloc[:, 0:2000]
y = cleaned['uw_index']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ridge = Ridge()
ridge.fit(X_train, y_train)

Ridge()

In [58]:
y_pred = ridge.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

corr, _ = pearsonr(y_test, y_pred)
print(f"Correlation: {corr}")

MSE: 0.4140476319053988
Correlation: 0.11630353546721091


In [182]:
X_train_pca = PCA(n_components=100).fit_transform(X_train)
X_test_pca = PCA(n_components=100).fit_transform(X_test)

ridge = Ridge()
ridge.fit(X_train_pca, y_train)

y_pred = ridge.predict(X_test_pca)

mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

corr, _ = pearsonr(y_test, y_pred)
print(f"Correlation: {corr}")

MSE: 0.414081360242471
Correlation: 0.04099887062793888


## BERT

### Preprocessing

In [288]:
docs_bert = tweets.groupby('FIPS')['Tweet'].apply(list)

In [292]:
docs_bert

,Tweet
FIPS,
01015,"[@DaneWise 8, I gotta remember to DVR this Reg..."
01039,[Home finally and I'm going inside to cough up...
01049,"[@Coltonio Obviously., @KimFidler I wanna be a..."
01073,"[@TCCarter23 Now, post a regualr picture so we..."
01081,[Laying here not feeling well at all....also t...
...,...
55131,[@RobKardashian Emotional cheating might be wo...
55133,[http://tweetphoto.com/14078454 Meme Si on par...
56001,[Man...had crazy dreams last night...in need o...


In [293]:
model = SentenceTransformer("all-MiniLM-L6-v2")

encoded_docs = {}

for key, doc in tqdm(docs_bert.items()):
  encoded_docs[key] = model.encode(doc)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

682it [29:11,  2.57s/it]


In [294]:
with open('encoded_docs.pickle', 'wb') as handle:
  pickle.dump(encoded_docs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
with open('encoded_docs.pickle', 'rb') as handle:
  encoded_docs = pickle.load(handle)

In [6]:
for key, doc in encoded_docs.items():
  encoded_docs[key] = np.mean(doc, axis=0)

In [7]:
encoded_docs_df = pd.DataFrame.from_dict(encoded_docs, orient='index')

In [304]:
encoded_docs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
01015,-0.031048,-0.011390,0.019395,-0.011752,0.014029,-0.006824,0.056717,-0.015721,0.022438,-0.006695,...,0.039294,-0.001411,0.012889,-0.010839,-0.005933,0.016765,0.034060,-0.019332,-0.041151,0.004886
01039,-0.028559,-0.030424,0.040171,-0.003805,0.035877,-0.002948,0.015763,-0.021747,0.003610,-0.002795,...,0.023499,0.001255,0.008101,-0.010249,0.001352,0.026917,0.053927,-0.018600,-0.071958,0.022865
01049,-0.030181,-0.005715,0.014483,-0.017501,0.022858,-0.006955,0.051375,0.000482,0.025814,-0.003739,...,0.039729,0.007428,0.009777,-0.016690,-0.003056,0.012840,0.039004,-0.017023,-0.034690,0.016084
01073,-0.028584,-0.001331,0.024173,-0.003109,0.022728,-0.002099,0.050749,-0.014577,0.016414,-0.001612,...,0.035498,-0.001013,0.017809,-0.015349,-0.001744,0.011586,0.043034,-0.015129,-0.037946,0.002796
01081,-0.034406,-0.008672,0.032349,-0.008390,0.010732,-0.013776,0.060172,-0.014474,0.023708,-0.014978,...,0.028375,0.006261,0.009089,-0.003405,-0.006072,0.016438,0.050782,-0.014463,-0.046077,-0.007289


In [9]:
encoded_docs_df['FIPS'] = encoded_docs_df.index

In [13]:
encoded_docs_df = encoded_docs_df.merge(hwbi[['FIPS', 'uw_index']], on='FIPS', how='left')

In [14]:
encoded_docs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,376,377,378,379,380,381,382,383,FIPS,uw_index
0,-0.031048,-0.011390,0.019395,-0.011752,0.014029,-0.006824,0.056717,-0.015721,0.022438,-0.006695,...,0.012889,-0.010839,-0.005933,0.016765,0.034060,-0.019332,-0.041151,0.004886,01015,1.286671
1,-0.028559,-0.030424,0.040171,-0.003805,0.035877,-0.002948,0.015763,-0.021747,0.003610,-0.002795,...,0.008101,-0.010249,0.001352,0.026917,0.053927,-0.018600,-0.071958,0.022865,01039,-0.490387
2,-0.030181,-0.005715,0.014483,-0.017501,0.022858,-0.006955,0.051375,0.000482,0.025814,-0.003739,...,0.009777,-0.016690,-0.003056,0.012840,0.039004,-0.017023,-0.034690,0.016084,01049,0.982228
3,-0.028584,-0.001331,0.024173,-0.003109,0.022728,-0.002099,0.050749,-0.014577,0.016414,-0.001612,...,0.017809,-0.015349,-0.001744,0.011586,0.043034,-0.015129,-0.037946,0.002796,01073,-0.088531
4,-0.034406,-0.008672,0.032349,-0.008390,0.010732,-0.013776,0.060172,-0.014474,0.023708,-0.014978,...,0.009089,-0.003405,-0.006072,0.016438,0.050782,-0.014463,-0.046077,-0.007289,01081,0.315359


### Model Training & Evaluation

In [153]:
cleaned = encoded_docs_df.dropna(subset=['uw_index'])

X = cleaned.iloc[:, 0:384]
y = cleaned['uw_index']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_pca = PCA(n_components=100).fit_transform(X_train)
X_test_pca = PCA(n_components=100).fit_transform(X_test)

In [145]:
# Define models

ridge = Ridge()
forest = RandomForestRegressor()

ridge_pca = Ridge()
forest_pca = RandomForestRegressor()

In [148]:
ridge.fit(X_train, y_train)
forest.fit(X_train, y_train)

ridge_pca.fit(X_train_pca, y_train)
forest_pca.fit(X_train_pca, y_train)

y_pred_ridge = ridge.predict(X_test)
y_pred_forest = forest.predict(X_test)

y_pred_ridge_pca = ridge_pca.predict(X_test_pca)
y_pred_forest_pca = forest_pca.predict(X_test_pca)

In [157]:
print(f"MSE Ridge: {mean_squared_error(y_test, y_pred_ridge)}, Correlation: {pearsonr(y_test, y_pred_ridge)[0]}")
print(f"MSE Forest: {mean_squared_error(y_test, y_pred_forest)}, Correlation: {pearsonr(y_test, y_pred_forest)[0]}")
print(f"MSE Ridge PCA: {mean_squared_error(y_test, y_pred_ridge_pca)}, Correlation: {pearsonr(y_test, y_pred_ridge_pca)[0]}")
print(f"MSE Forest PCA: {mean_squared_error(y_test, y_pred_forest_pca)}, Correlation: {pearsonr(y_test, y_pred_forest_pca)[0]}")

MSE Ridge: 0.4494421323731345, Correlation: 0.1781978594771877
MSE Forest: 0.43601386877233705, Correlation: 0.21327011631873036
MSE Ridge PCA: 0.467175196371735, Correlation: -0.2139425725282534
MSE Forest PCA: 0.4696311328141181, Correlation: 0.0005792628731210669
